# DLR Dataset Timeseries Description

In [ ]:
import pandas as pd
import numpy as np
import datetime as dt

import plotly.plotly as py
import plotly.offline as po
import plotly.graph_objs as go
import plotly.tools as tls
import colorlover as cl
#from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import cufflinks as cf
cf.go_offline()

from features.feature_ts import *
from observations.obs_processing import *
from experiment.algorithms.clusters import preprocessX

### Retrieve Profiles

In [ ]:
year_range = [1994,2014]
X = genX(year_range)

## Household Count

In [ ]:
dailycount = X.groupby(pd.Grouper(freq='D', level=-1))['0'].count()
monthlycount = dailycount.resample('M').agg(np.median)
med_annualcount = dailycount.resample('A').agg(np.median)
max_annualcount = dailycount.resample('A').agg(np.max)
min_annualcount = dailycount.resample('A').agg(np.min)
profilecount = X.groupby(pd.Grouper(level=0))['0'].count()

In [ ]:
print('median daily household count: ', dailycount.median())
print('median monthly household count: ', monthlycount.median())
print('median annual household count: ', med_annualcount.median())
print('total daily load profiles from 1994 - 2014: ', dailycount.sum())
print('total number of households observed: ', len(profilecount))
print('households observed for half a year or more: ', len(profilecount[profilecount/365>0.5]))
print('mean days observed per household: ', int(profilecount.mean()))

In [ ]:
dailycount[dailycount == 1245]

### Count of Households Observed / Year

In [ ]:
annualcount = pd.concat([max_annualcount, med_annualcount, min_annualcount], axis=1, keys=['max','median','min'])
annualcount.head()

In [ ]:
trace1 = go.Bar(dict(
    x=annualcount.index, 
    y=annualcount['max'],
    marker = dict(color='purple')
))

trace2 = go.Bar(dict(
    x=annualcount.index, 
    y=annualcount['median'],
    marker = dict(color='red')
))

trace3 = go.Bar(dict(
    x=annualcount.index, 
    y=annualcount['min'],
    marker = dict(color='orange')
))

layout = go.Layout(dict(
    xaxis = dict(title='Year'),
    yaxis = dict(title='number of households'),
    barmode='stack'
))

data = [trace3, trace2, trace1]

po.plot(data)

#cufflinks that no longer works
#annualcount.iplot(kind='bar', xTitle='Year', yTitle='number of households', colors=['blue','red','orange'],
#                  title='Min, median and max household count observered per day per year',
#                barmode='overlay')

### Count of Households Observed / Day

In [ ]:
trace1 = go.Bar(dict(
    x=dailycount.index, 
    y=dailycount.values,
    marker = dict(color='darkblue')
))

layout = go.Layout(dict(
    title='Number of households observed for each day from 1994 - 2014',
    xaxis = dict(title='Year'),
    yaxis = dict(title='number of households'),
    barmode='stack'
))

data = [trace1]

po.plot(dict(data=data, layout=layout))

#dailycount.iplot(kind='bar', xTitle='Year', yTitle='number of households', 
#                 title='Number of households observed for each day from 1994 - 2014')

### Count of Days Observed 

In [ ]:
profilecount.iplot(kind='histogram', xTitle='number of days observed (shaded area indicates more than half a year observed)', 
                   yTitle='number of IDs', colors='red',
                   title = 'Count of Profile IDs observed for n days',
                   vspan={'x0':182,'x1':profilecount.max(),'color':'grey','fill':True,'opacity':.05})

## Average Demand

### Daily Demand (kWh)

In [ ]:
Xdd_A = X.sum(axis=1)
#Xdd_A.iplot(kind='histogram', bins=100, histnorm='percent', title='Histogram of daily demand of all profiles',
#          yTitle='% of profiles in bin',xTitle='binned daily demand (A)',color='blue')

In [ ]:
Xdd = Xdd_A*230/1000

### Average Annual Daily Demand (A)

In [ ]:
Xadd = resampleProfiles(Xdd,'A','mean').reset_index().groupby('ProfileID').mean()
Xadd.columns = ['ADD_kWh']
#Xadd.iplot(kind='histogram', bins=100, histnorm='percent', title='Histogram of average annual daily demand of all profiles',
#          yTitle='% of profiles in bin',xTitle='bins of average daily consumption (kWh)')

In [ ]:
Xadd.head()

In [ ]:
trace1 = go.Histogram(dict(
    x=Xadd['ADD_kWh'], 
    nbinsx=200,
    marker=dict(color='purple')
))

layout = go.Layout(dict(
    title='Histogram of annualised average daily energy consumption of all households',
    font=dict(size=22),
    xaxis = dict(title='average daily consumption bins (kWh)', dtick = 10, titlefont=dict(size=20), tickfont=dict(size=18)),
    yaxis = dict(title='count of profiles in bin', titlefont=dict(size=20), tickfont=dict(size=18))
))

data = [trace1]

po.plot(dict(data=data, layout=layout))

### Average Monthly Demand (kWh)

In [ ]:
XmonthlyPower = resampleProfiles(Xdd, interval='M', aggfunc='sum')
Xamd = resampleProfiles(XmonthlyPower, interval='A', aggfunc='mean').reset_index().groupby('ProfileID').mean()
Xamd.columns = ['AMD_kWh']
Xamd.iplot(kind='histogram', bins=100, histnorm='percent', 
           title='Histogram of average monthly energy consumption for households',
           yTitle='% of households in bin',xTitle='bins of average monthly consumption (kWh)')

### Representative Load Profiles

In [ ]:
Xrlp = resampleProfiles(X,'A','mean').reset_index().groupby('ProfileID').mean()
Xrlp.head()

In [ ]:
bins = pd.cut(Xamd.iloc[:,0], range(0, 100*int(np.ceil(Xamd.max()/100))+1, 100 ), labels=range(100, 100*int(np.ceil(Xamd.max()/100))+1, 100 ))
amdrlp = Xrlp.join(bins)
MeanRLP = amdrlp.groupby('AMD (kWh)').mean().fillna(0)
StdRLP = Xrlp.join(bins).groupby('AMD (kWh)').std().fillna(0)

def plotRLP(maxMonthlyDemand, color='red'):
    amdrlp.loc[amdrlp['AMD (kWh)'] == maxMonthlyDemand, amdrlp.columns!='AMD (kWh)'].iplot(kind='box', 
            title='Variance of mean hourly demand for households with an average monthly demand between ' + str(maxMonthlyDemand-100) + ' and '+ str(maxMonthlyDemand) + ' kWh',
            xTitle = 'time of day',
            yTitle = 'mean hourly demand (kWh)',
            color=color,
            legend=False)
                                                                                           
plotRLP(100, 'blue')
    
MeanRLP.T.iplot(kind='scatter', 
            title='Representative load profiles for households in the same range of average monthly consumption', 
            xTitle = 'time of day',
            yTitle = 'mean hourly demand (kWh)',
            width = 3,)


### Experimenting with Self Organising Maps & kmeans

In [ ]:
som.view_umatrix(colorbar=True, bestmatches=True)

In [ ]:
som.view_component_planes([6])

In [ ]:
pd.DataFrame(som.clusters).iplot(kind='heatmap')

In [ ]:
k = [som.clusters[som.bmus[i][1],som.bmus[i][0]] for i in range(0, len(som.bmus))]
pd.DataFrame(k).iplot(kind='histogram', title='count of profiles per cluster kmeans + SOM')

In [ ]:
len(som.codebook)
#som.n_dim
#pd.DataFrame(som.codebook).groupby

m = np.arange(0, nrow*ncol, 1).reshape(nrow, ncol)
k = [m[som.bmus[i][1],som.bmus[i][0]] for i in range(0, len(som.bmus))]
c = som.codebook.reshape(nrow * ncol, som.n_dim)

In [ ]:
pd.DataFrame(clustX).iplot(kind='histogram', title='count of profiles per cluster kmeans only', color='blue')